  Author: Ankit Kariryaa, University of Bremen
  
  Modified by Xuehui Pi and Qiuqi Luo

### Getting started
Define the paths for the dataset and trained models in the `notebooks/config/UNetTraining.py` file.  

In [1]:
import os
os.environ["MKL_NUM_THREADS"] = '16'
os.environ["NUMEXPR_NUM_THREADS"] = '16'
os.environ["OMP_NUM_THREADS"] = '16'
print(os.environ.get('OMP_NUM_THREADS'))

16


In [2]:
import tensorflow as tf
import numpy as np
from PIL import Image
import rasterio
import imgaug as ia
from imgaug import augmenters as iaa
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras import mixed_precision 
mixed_precision.set_global_policy('mixed_float16')

import imageio
import os

import time
import rasterio.warp             # Reproject raster samples
from functools import reduce
from tensorflow.keras.models import load_model

from core.UNet_572 import UNet  #
from core.losses import tversky, focalTversky, bce_dice_loss, accuracy, dice_loss, IoU, recall, precision,F1_score
from tensorflow.keras.losses import BinaryCrossentropy as bce
from core.optimizers import adaDelta, adagrad, adam, nadam
from core.frame_info_572 import FrameInfo
from core.dataset_generator_572 import DataGenerator
from core.visualize import display_images,plot
import json
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import pickle
import random

%matplotlib inline
import matplotlib.pyplot as plt  # plotting tools
import matplotlib.patches as patches
from matplotlib.patches import Polygon
#matplotlib.use("Agg")

import warnings                  # ignore annoying warnings
warnings.filterwarnings("ignore")
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

%reload_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


#Mixed precision is the use of both 16-bit and 32-bit floating-point types in a model during training to make it run faster and use less memory.
os.environ['TF_ENABLE_AUTO_MIXED_PRECISION'] = '1'
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"
print(tf.__version__)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3090, compute capability 8.6
2.9.0


In [3]:
# print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
# tf.device('/gpu:1')

In [4]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto(
    #device_count={"CPU": 64},
    allow_soft_placement=True, 
    log_device_placement=False)
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [5]:
# Required configurations (including the input and output paths) are stored in a separate file (such as config/UNetTraining.py)
# Please provide required info in the file before continuing with this notebook. 
# hbh: in this scene,a new config named UNetTraining_sequential is created to distinguish from the original
from config import UNetTraining_572
# In case you are using a different folder name such as configLargeCluster, then you should import from the respective folder 
# Eg. from configLargeCluster import UNetTraining
config = UNetTraining_572.Configuration()

## 参数初始化

In [6]:
OPTIMIZER = adaDelta
OPTIMIZER =  mixed_precision.LossScaleOptimizer(OPTIMIZER)
OPTIMIZER_NAME = 'AdaDelta'

# OPTIMIZER = adam
# OPTIMIZER = mixed_precision.LossScaleOptimizer(OPTIMIZER)
# OPTIMIZER_NAME = 'adam'

In [7]:
# LOSS = tversky 
# LOSS_NAME = 'tversky'

# LOSS=focalTversky
# LOSS_NAME = 'focalTversky'

#LOSS=tf.keras.losses.BinaryCrossentropy()
#LOSS_NAME = 'bce'

# LOSS=bce_dice_loss
# LOSS_NAME = 'bce_dice_loss'

LOSS=dice_loss
LOSS_NAME = 'dice_loss'


In [8]:
def readImgs(path_to_write, fn):
    image = rasterio.open(os.path.join(path_to_write, fn))
#     print(fn)
    read_image = image.read()/1000
#     print(read_image[1].max())
    
    comb_img = np.transpose(read_image, axes=(1,2,0))
    annotation_im = Image.open(os.path.join(path_to_write, fn.replace(config.image_fn,config.annotation_fn).replace(config.image_type,config.ann_type)))
    annotation = np.array(annotation_im)
    rowNum=annotation.shape[0]/config.output_size[0]#config.patch_size[0]
    colNum=annotation.shape[1]/config.output_size[1]#config.patch_size[1]
#     if(read_image.shape[1]- annotation.shape[0]!=184) or(read_image.shape[2]- annotation.shape[1]!=184):
#                 print(fn)
    f = FrameInfo(comb_img, annotation)
    return f,rowNum*colNum
    
def readFrames(dataType):
    frames=[]
    numList=[]
    print(dataType)
#     for i in range(3,4):
    for i in range(0,config.type_num):#config.type_num
        path_to_write=os.path.join(config.dataset_dir,'{}/type{}'.format(dataType,i))
        all_files = os.listdir(path_to_write)
        all_files_image = [fn for fn in all_files if fn.startswith(config.image_fn) and fn.endswith(config.image_type)]#image.png
        print('type{} image number:{}'.format(i,len(all_files_image)))
        for j, fn in enumerate(all_files_image):
            f,num = readImgs(path_to_write,fn)
            frames.append(f)
            numList.append(num)
    return frames,numList


## 数据集准备

In [9]:
frames,numList=readFrames('train')
percentages=np.array(numList)
train_sum=percentages.sum()
percentages=percentages/percentages.sum()
print('total training img count:'+str(len(frames)))
print('total training patches count:{}'.format(train_sum))
train_generator = DataGenerator(config.input_size, config.output_size,frames, augmenter = 'iaa').random_generator(config.BATCH_SIZE,percentages)#,normalize =

train
type0 image number:156
type1 image number:141
type2 image number:68
type3 image number:53
type4 image number:9
type5 image number:17
total training img count:444
total training patches count:17414.319155861413


In [ ]:
# numList=readNum('train')
# percentages=np.array(numList)
# print(percentages.sum())

In [10]:
frames,numList=readFrames('val')
percentages=np.array(numList)
val_sum=percentages.sum()
percentages=percentages/percentages.sum()
print('total validation img count:'+str(len(frames)))
print('total validation patches count:{}'.format(val_sum))
val_generator = DataGenerator(config.input_size, config.output_size, frames, augmenter = None).random_generator(config.BATCH_SIZE,percentages)#, normalize = config.normalize

val
type0 image number:53
type1 image number:48
type2 image number:24
type3 image number:18
type4 image number:3
type5 image number:5
total validation img count:151
total validation patches count:5719.94284063131


In [12]:
del frames

In [ ]:
for _ in range(1):
    train_images, real_label = next(train_generator) 
    print(train_images.shape)
    print(real_label.shape)
    display_images(train_images,real_label,pad=92,output_size=config.output_size[0])

In [ ]:
for _ in range(1):
    val_images, val_label = next(val_generator) 
    print(val_images.shape)
    display_images(val_images,val_label,pad=92,output_size=config.output_size[0])

### 训练参数设置

In [11]:
timestr = time.strftime("%Y%m%d-%H%M")
chf = config.input_image_channel + config.input_label_channel
chs = reduce(lambda a,b: a+str(b), chf, '') 

if not os.path.exists(config.model_path):
    os.makedirs(config.model_path)
model_name='_{}_{}_{}_{}_{}.h5'.format(timestr,OPTIMIZER_NAME,LOSS_NAME,chs,config.input_shape[0])
model_path = os.path.join(config.model_path,'lakes'+model_name)
chf = config.input_image_channel + config.input_label_channel
chs = reduce(lambda a,b: a+str(b), chf, '') 
print(model_path)

G:\lakemapping\5_saved_models\lakes_20240112-1629_AdaDelta_dice_loss_012345_572.h5


In [12]:
# Define the model and compile it  
print(config.input_shape)
shape=[config.BATCH_SIZE, *config.input_shape]
model = UNet([config.BATCH_SIZE, *config.input_shape],config.input_label_channel)
model.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=[dice_loss, accuracy, recall, precision,F1_score, IoU])

(572, 572, 5)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input (InputLayer)             [(None, 572, 572, 5  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 570, 570, 64  2944        ['Input[0][0]']                  
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 568, 568, 64  36928       ['conv2d[0][0]']                 
                                )                                               

 conv2d_12 (Conv2D)             (None, 102, 102, 25  1769728     ['concatenate_1[0][0]']          
                                6)                                                                
                                                                                                  
 conv2d_13 (Conv2D)             (None, 100, 100, 25  590080      ['conv2d_12[0][0]']              
                                6)                                                                
                                                                                                  
 up_sampling2d_2 (UpSampling2D)  (None, 200, 200, 25  0          ['conv2d_13[0][0]']              
                                6)                                                                
                                                                                                  
 batch_normalization_6 (BatchNo  (None, 200, 200, 25  1024       ['up_sampling2d_2[0][0]']        
 rmalizati

In [13]:
# Define callbacks      for the early stopping of training, LearningRateScheduler and model checkpointing 
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard

checkpoint = ModelCheckpoint(model_path, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = False)

#reduceonplatea： It can be useful when using adam as optimizer
#Reduce learning rate when a metric has stopped improving (after some patience 个epoch, reduce by a factor of 0.33, new_lr = lr * factor). 
#cooldown: number of epochs to wait before resuming normal operation after lr has been reduced. 

reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.33,
                                   patience=4, verbose=1, mode='min',
                                   min_delta=0.0001, cooldown=4, min_lr=1e-16) 

early = EarlyStopping(monitor="val_loss", mode="min", verbose=2, patience=40)


log_dir = os.path.join('./logs','UNet'+model_name)
tensorboard = TensorBoard(log_dir=log_dir, histogram_freq=0, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None, update_freq='epoch')

callbacks_list = [checkpoint, tensorboard] #reduceLROnPlat is not required with adaDelta, early

## 训练模型
保存history文件并绘图

In [14]:
loss_history = model.fit(train_generator, 
                         steps_per_epoch=train_sum//config.BATCH_SIZE,#config.MAX_TRAIN_STEPS,
                         epochs=config.NB_EPOCHS, 
                         validation_data=val_generator,
                         validation_steps=val_sum//config.BATCH_SIZE,#config.VALID_IMG_COUNT,
                         callbacks=callbacks_list,
                         workers=1,
                         # shuffle=True,
#                          use_multiprocessing=True # the generator is not very thread safe 
                         #max_queue_size = 60,
                        )
h=loss_history.history
with open('history_{}_{}_{}_{}_{}.txt'.format(timestr,OPTIMIZER_NAME,LOSS_NAME, chs,config.input_shape[0]), 'wb') as file_pi:
    pickle.dump(h, file_pi)
plot(h,timestr, OPTIMIZER_NAME,LOSS_NAME, config.input_size[0], config.NB_EPOCHS, config.BATCH_SIZE,chs)

Epoch 1/150
1088/1088 [==============================] - ETA: 0s - loss: 0.2519 - dice_loss: 0.2517 - accuracy: 0.9843 - recall: 0.8386 - precision: 0.7102 - F1_score: 0.7490 - IoU: 0.6229
Epoch 1: val_loss improved from inf to 0.29936, saving model to G:\lakemapping\5_saved_models\lakes_20240112-1629_AdaDelta_dice_loss_012345_572.h5
1088/1088 [==============================] - 707s 599ms/step - loss: 0.2519 - dice_loss: 0.2517 - accuracy: 0.9843 - recall: 0.8386 - precision: 0.7102 - F1_score: 0.7490 - IoU: 0.6229 - val_loss: 0.2994 - val_dice_loss: 0.2992 - val_accuracy: 0.9846 - val_recall: 0.6672 - val_precision: 0.7755 - val_F1_score: 0.7009 - val_IoU: 0.5614
Epoch 2/150
1088/1088 [==============================] - ETA: 0s - loss: 0.2115 - dice_loss: 0.2113 - accuracy: 0.9878 - recall: 0.8650 - precision: 0.7511 - F1_score: 0.7889 - IoU: 0.6713
Epoch 2: val_loss improved from 0.29936 to 0.18207, saving model to G:\lakemapping\5_saved_models\lakes_20240112-1629_AdaDelta_dice_loss_0

1088/1088 [==============================] - ETA: 0s - loss: 0.0875 - dice_loss: 0.0874 - accuracy: 0.9955 - recall: 0.9152 - precision: 0.9178 - F1_score: 0.9127 - IoU: 0.8452
Epoch 14: val_loss did not improve from 0.09890
1088/1088 [==============================] - 669s 614ms/step - loss: 0.0875 - dice_loss: 0.0874 - accuracy: 0.9955 - recall: 0.9152 - precision: 0.9178 - F1_score: 0.9127 - IoU: 0.8452 - val_loss: 0.0998 - val_dice_loss: 0.0997 - val_accuracy: 0.9942 - val_recall: 0.8994 - val_precision: 0.9123 - val_F1_score: 0.9003 - val_IoU: 0.8291
Epoch 15/150
1088/1088 [==============================] - ETA: 0s - loss: 0.0856 - dice_loss: 0.0855 - accuracy: 0.9956 - recall: 0.9167 - precision: 0.9195 - F1_score: 0.9146 - IoU: 0.8483
Epoch 15: val_loss did not improve from 0.09890
1088/1088 [==============================] - 664s 610ms/step - loss: 0.0856 - dice_loss: 0.0855 - accuracy: 0.9956 - recall: 0.9167 - precision: 0.9195 - F1_score: 0.9146 - IoU: 0.8483 - val_loss: 0.1

Epoch 28/150
1088/1088 [==============================] - ETA: 0s - loss: 0.0708 - dice_loss: 0.0708 - accuracy: 0.9963 - recall: 0.9312 - precision: 0.9322 - F1_score: 0.9293 - IoU: 0.8720
Epoch 28: val_loss did not improve from 0.09532
1088/1088 [==============================] - 666s 612ms/step - loss: 0.0708 - dice_loss: 0.0708 - accuracy: 0.9963 - recall: 0.9312 - precision: 0.9322 - F1_score: 0.9293 - IoU: 0.8720 - val_loss: 0.1253 - val_dice_loss: 0.1252 - val_accuracy: 0.9925 - val_recall: 0.8572 - val_precision: 0.9109 - val_F1_score: 0.8748 - val_IoU: 0.7911
Epoch 29/150
1088/1088 [==============================] - ETA: 0s - loss: 0.0645 - dice_loss: 0.0645 - accuracy: 0.9966 - recall: 0.9350 - precision: 0.9400 - F1_score: 0.9356 - IoU: 0.8823
Epoch 29: val_loss did not improve from 0.09532
1088/1088 [==============================] - 665s 611ms/step - loss: 0.0645 - dice_loss: 0.0645 - accuracy: 0.9966 - recall: 0.9350 - precision: 0.9400 - F1_score: 0.9356 - IoU: 0.8823 - 

Epoch 42/150
1088/1088 [==============================] - ETA: 0s - loss: 0.0574 - dice_loss: 0.0574 - accuracy: 0.9970 - recall: 0.9432 - precision: 0.9453 - F1_score: 0.9427 - IoU: 0.8947
Epoch 42: val_loss did not improve from 0.08783
1088/1088 [==============================] - 666s 611ms/step - loss: 0.0574 - dice_loss: 0.0574 - accuracy: 0.9970 - recall: 0.9432 - precision: 0.9453 - F1_score: 0.9427 - IoU: 0.8947 - val_loss: 0.1063 - val_dice_loss: 0.1063 - val_accuracy: 0.9942 - val_recall: 0.9005 - val_precision: 0.9021 - val_F1_score: 0.8938 - val_IoU: 0.8211
Epoch 43/150
1088/1088 [==============================] - ETA: 0s - loss: 0.0574 - dice_loss: 0.0574 - accuracy: 0.9970 - recall: 0.9420 - precision: 0.9460 - F1_score: 0.9426 - IoU: 0.8939
Epoch 43: val_loss did not improve from 0.08783
1088/1088 [==============================] - 663s 609ms/step - loss: 0.0574 - dice_loss: 0.0574 - accuracy: 0.9970 - recall: 0.9420 - precision: 0.9460 - F1_score: 0.9426 - IoU: 0.8939 - 

1088/1088 [==============================] - ETA: 0s - loss: 0.0521 - dice_loss: 0.0520 - accuracy: 0.9974 - recall: 0.9481 - precision: 0.9499 - F1_score: 0.9480 - IoU: 0.9033
Epoch 56: val_loss did not improve from 0.08277
1088/1088 [==============================] - 669s 615ms/step - loss: 0.0521 - dice_loss: 0.0520 - accuracy: 0.9974 - recall: 0.9481 - precision: 0.9499 - F1_score: 0.9480 - IoU: 0.9033 - val_loss: 0.0913 - val_dice_loss: 0.0913 - val_accuracy: 0.9947 - val_recall: 0.8920 - val_precision: 0.9371 - val_F1_score: 0.9088 - val_IoU: 0.8428
Epoch 57/150
1088/1088 [==============================] - ETA: 0s - loss: 0.0527 - dice_loss: 0.0527 - accuracy: 0.9973 - recall: 0.9469 - precision: 0.9500 - F1_score: 0.9473 - IoU: 0.9022
Epoch 57: val_loss did not improve from 0.08277
1088/1088 [==============================] - 664s 610ms/step - loss: 0.0527 - dice_loss: 0.0527 - accuracy: 0.9973 - recall: 0.9469 - precision: 0.9500 - F1_score: 0.9473 - IoU: 0.9022 - val_loss: 0.1

...

1088/1088 [==============================] - ETA: 0s - loss: 0.0435 - dice_loss: 0.0435 - accuracy: 0.9978 - recall: 0.9564 - precision: 0.9584 - F1_score: 0.9565 - IoU: 0.9185
Epoch 84: val_loss did not improve from 0.07659
1088/1088 [==============================] - 661s 607ms/step - loss: 0.0435 - dice_loss: 0.0435 - accuracy: 0.9978 - recall: 0.9564 - precision: 0.9584 - F1_score: 0.9565 - IoU: 0.9185 - val_loss: 0.0940 - val_dice_loss: 0.0939 - val_accuracy: 0.9952 - val_recall: 0.8967 - val_precision: 0.9314 - val_F1_score: 0.9061 - val_IoU: 0.8410
Epoch 85/150
1088/1088 [==============================] - ETA: 0s - loss: 0.0456 - dice_loss: 0.0456 - accuracy: 0.9977 - recall: 0.9540 - precision: 0.9562 - F1_score: 0.9545 - IoU: 0.9145
Epoch 85: val_loss did not improve from 0.07659
1088/1088 [==============================] - 659s 605ms/step - loss: 0.0456 - dice_loss: 0.0456 - accuracy: 0.9977 - recall: 0.9540 - precision: 0.9562 - F1_score: 0.9545 - IoU: 0.9145 - val_loss: 0.0

1088/1088 [==============================] - ETA: 0s - loss: 0.0428 - dice_loss: 0.0427 - accuracy: 0.9979 - recall: 0.9562 - precision: 0.9597 - F1_score: 0.9573 - IoU: 0.9194
Epoch 98: val_loss did not improve from 0.07659
1088/1088 [==============================] - 656s 603ms/step - loss: 0.0428 - dice_loss: 0.0427 - accuracy: 0.9979 - recall: 0.9562 - precision: 0.9597 - F1_score: 0.9573 - IoU: 0.9194 - val_loss: 0.0863 - val_dice_loss: 0.0863 - val_accuracy: 0.9952 - val_recall: 0.9116 - val_precision: 0.9251 - val_F1_score: 0.9137 - val_IoU: 0.8510
Epoch 99/150
1088/1088 [==============================] - ETA: 0s - loss: 0.0431 - dice_loss: 0.0431 - accuracy: 0.9978 - recall: 0.9565 - precision: 0.9590 - F1_score: 0.9570 - IoU: 0.9191
Epoch 99: val_loss did not improve from 0.07659
1088/1088 [==============================] - 651s 599ms/step - loss: 0.0431 - dice_loss: 0.0431 - accuracy: 0.9978 - recall: 0.9565 - precision: 0.9590 - F1_score: 0.9570 - IoU: 0.9191 - val_loss: 0.0

1088/1088 [==============================] - ETA: 0s - loss: 0.0392 - dice_loss: 0.0392 - accuracy: 0.9980 - recall: 0.9597 - precision: 0.9630 - F1_score: 0.9608 - IoU: 0.9257
Epoch 112: val_loss did not improve from 0.07335
1088/1088 [==============================] - 664s 610ms/step - loss: 0.0392 - dice_loss: 0.0392 - accuracy: 0.9980 - recall: 0.9597 - precision: 0.9630 - F1_score: 0.9608 - IoU: 0.9257 - val_loss: 0.0851 - val_dice_loss: 0.0851 - val_accuracy: 0.9952 - val_recall: 0.9024 - val_precision: 0.9387 - val_F1_score: 0.9150 - val_IoU: 0.8522
Epoch 113/150
1088/1088 [==============================] - ETA: 0s - loss: 0.0384 - dice_loss: 0.0383 - accuracy: 0.9980 - recall: 0.9609 - precision: 0.9633 - F1_score: 0.9617 - IoU: 0.9272
Epoch 113: val_loss did not improve from 0.07335
1088/1088 [==============================] - 662s 608ms/step - loss: 0.0384 - dice_loss: 0.0383 - accuracy: 0.9980 - recall: 0.9609 - precision: 0.9633 - F1_score: 0.9617 - IoU: 0.9272 - val_loss: 

1088/1088 [==============================] - ETA: 0s - loss: 0.0371 - dice_loss: 0.0371 - accuracy: 0.9981 - recall: 0.9629 - precision: 0.9638 - F1_score: 0.9629 - IoU: 0.9294
Epoch 126: val_loss did not improve from 0.07294
1088/1088 [==============================] - 663s 609ms/step - loss: 0.0371 - dice_loss: 0.0371 - accuracy: 0.9981 - recall: 0.9629 - precision: 0.9638 - F1_score: 0.9629 - IoU: 0.9294 - val_loss: 0.0924 - val_dice_loss: 0.0924 - val_accuracy: 0.9954 - val_recall: 0.8891 - val_precision: 0.9428 - val_F1_score: 0.9077 - val_IoU: 0.8432
Epoch 127/150
1088/1088 [==============================] - ETA: 0s - loss: 0.0373 - dice_loss: 0.0373 - accuracy: 0.9981 - recall: 0.9626 - precision: 0.9636 - F1_score: 0.9627 - IoU: 0.9290
Epoch 127: val_loss did not improve from 0.07294
1088/1088 [==============================] - 664s 610ms/step - loss: 0.0373 - dice_loss: 0.0373 - accuracy: 0.9981 - recall: 0.9626 - precision: 0.9636 - F1_score: 0.9627 - IoU: 0.9290 - val_loss: 

1088/1088 [==============================] - ETA: 0s - loss: 0.0340 - dice_loss: 0.0340 - accuracy: 0.9983 - recall: 0.9648 - precision: 0.9680 - F1_score: 0.9661 - IoU: 0.9352
Epoch 140: val_loss did not improve from 0.07022
1088/1088 [==============================] - 647s 595ms/step - loss: 0.0340 - dice_loss: 0.0340 - accuracy: 0.9983 - recall: 0.9648 - precision: 0.9680 - F1_score: 0.9661 - IoU: 0.9352 - val_loss: 0.0822 - val_dice_loss: 0.0821 - val_accuracy: 0.9952 - val_recall: 0.9135 - val_precision: 0.9336 - val_F1_score: 0.9179 - val_IoU: 0.8578
Epoch 141/150
1088/1088 [==============================] - ETA: 0s - loss: 0.0351 - dice_loss: 0.0351 - accuracy: 0.9982 - recall: 0.9645 - precision: 0.9662 - F1_score: 0.9649 - IoU: 0.9332
Epoch 141: val_loss did not improve from 0.07022
1088/1088 [==============================] - 655s 602ms/step - loss: 0.0351 - dice_loss: 0.0351 - accuracy: 0.9982 - recall: 0.9645 - precision: 0.9662 - F1_score: 0.9649 - IoU: 0.9332 - val_loss: 

In [ ]:
##### 读取现有history文件
with open('.txt','rb')as file_pi: 
    h=pickle.load(file_pi)
print(h)

In [17]:
plot(h, OPTIMIZER_NAME,LOSS_NAME, config.patch_size[0], config.NB_EPOCHS, config.BATCH_SIZE,chs)

## 模型精度评价

In [9]:
# Load model after training 
model_path=os.path.join(config.model_path,'lakes_20240112-1629_AdaDelta_dice_loss_012345_572.h5')
model = load_model(model_path, custom_objects={'dice loss': LOSS, 'accuracy':accuracy ,'recall':recall, 'F1_score':F1_score,'precision':precision,'IoU': IoU}, compile=False) 
model.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=[dice_loss, accuracy,recall,F1_score, precision, IoU])

In [10]:
# # Print one batch on the training/test data! 
# for i in range(1):
#     test_images, real_label = next(test_generator)
#     #3 images per row: GSW, label, prediction
#     prediction = model.predict(test_images, steps=1)
#     prediction[prediction>0.5]=1
#     prediction[prediction<=0.5]=0
#     display_images(np.concatenate((test_images, real_label, prediction), axis = -1))# test_images( NDWI), real_label(label), prediction

In [ ]:
j=1
frames=[]
path_to_write=os.path.join(config.dataset_dir,'test/type'+str(j))
all_files = os.listdir(path_to_write)
all_image_files = [fn for fn in all_files if fn.startswith(config.image_fn) and fn.endswith(config.image_type)]#ndwi.png
for i, fn in enumerate(all_image_files):
    f,nums = readImgs(path_to_write,fn)
    frames.append(f)
test_DGT=DataGenerator(config.input_size, config.output_size,frames, augmenter = None)
tp_img,tp_ann= test_DGT.all_sequential_patches(config.step_size)
print(len(tp_ann))

titles=['ndwi','rgb','swir','annotation','prediction']
for i in range(len(tp_ann//config.BATCH_SIZE)+1):
    prediction = model.predict(tp_img[i*config.BATCH_SIZE:(i+1)*config.BATCH_SIZE], steps=1)
    prediction[prediction>0.5]=1
    prediction[prediction<=0.5]=0
    fn=os.path.join(config.base_dir,r'4_prediction\sample746\image\type{}_{}_{}.png'.format(j,i,config.output_size[0]))
    display_images(tp_img[i*config.BATCH_SIZE:(i+1)*config.BATCH_SIZE], tp_ann[i*config.BATCH_SIZE:(i+1)*config.BATCH_SIZE], prediction,92,config.output_size[0],fn.format(i),titles=titles)

## 总体精度评价

In [10]:
frames,numList=readFrames('test')
test_patches = DataGenerator(config.input_size, config.output_size, frames, augmenter = None).all_sequential_patches(config.step_size)
print('test patchs number:',len(test_patches[0]))

test
type0 image number:53
type1 image number:48
type2 image number:23
type3 image number:18
type4 image number:3
type5 image number:6
test patchs number: 5254


In [11]:
ev=model.evaluate(test_patches[0],test_patches[1],config.BATCH_SIZE)
ev.append(len(test_patches[0]))

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [19]:
print(ev)

[0.11038431525230408, 0.11386916786432266, 0.9946683049201965, 0.9123051166534424, 0.8933606743812561, 0.8861443400382996, 0.8190804719924927]


## 分类别精度评价

In [10]:
ev_list=[]
for i in range(1,config.type_num):
    frames=[]
    test_patchs_type=[]
    path_to_write=os.path.join(config.dataset_dir,'test/type'+str(i))
    all_files = os.listdir(path_to_write)
    all_image_files = [fn for fn in all_files if fn.startswith(config.image_fn) and fn.endswith(config.image_type)]#ndwi.png
    for j, fn in enumerate(all_image_files):
        f ,num = readImgs(path_to_write,fn)
        frames.append(f)
    test_patchs_type = DataGenerator(config.input_size, config.output_size, frames, augmenter = None).all_sequential_patches(config.step_size)
#     print('type{} patches number:{}'.format(i,len(test_patchs_type[0])))
    ev2=model.evaluate(test_patchs_type[0],test_patchs_type[1],config.BATCH_SIZE,verbose=1)
    ev2.append(len(test_patchs_type[0]))
    ev_list.append(ev2)

50/50 [==============================] - 9s 156ms/step - loss: 0.1029 - dice_loss: 0.1046 - accuracy: 0.9981 - recall: 0.9172 - F1_score: 0.8954 - precision: 0.8983 - IoU: 0.8251


ResourceExhaustedError: Graph execution error:

Detected at node 'model/conv2d/Conv2D' defined at (most recent call last):
    File "C:\Users\asus\anaconda3\envs\tf_gpu_38\lib\runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\asus\anaconda3\envs\tf_gpu_38\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\asus\AppData\Roaming\Python\Python38\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\asus\anaconda3\envs\tf_gpu_38\lib\site-packages\traitlets\config\application.py", line 1077, in launch_instance
      app.start()
    File "C:\Users\asus\AppData\Roaming\Python\Python38\site-packages\ipykernel\kernelapp.py", line 736, in start
      self.io_loop.start()
    File "C:\Users\asus\AppData\Roaming\Python\Python38\site-packages\tornado\platform\asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\as

In [14]:
ev_list=[]
ev_list.append(ev2)

In [17]:
ev2=model.evaluate(test_patchs_type[0],test_patchs_type[1],config.BATCH_SIZE,verbose=1)
ev2.append(len(test_patchs_type[0]))
ev_list.append(ev2)

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [16]:
print('      patch_num   loss  dice_loss accuracy   recall  precision F1_score     IoU')
# print('total: {:^9}{:^9.3f}{:^9.3f}{:^9.3f} {:^9.3f} {:^9.3f} {:^9.3f} {:^9.3f}'.format(ev[7],ev[0],ev[1],ev[2],ev[3],ev[4],ev[5],ev[6]))
for i in range(0,config.type_num):
    print('type{}:{:^9}{:^9.3f}{:^9.3f}{:^9.3f} {:^9.3f} {:^9.3f} {:^9.3f} {:^9.3f}'.format(i,ev_list[i][7],ev_list[i][0],ev_list[i][1],ev_list[i][2],ev_list[i][3],ev_list[i][4],ev_list[i][5],ev_list[i][6]))

      patch_num   loss  dice_loss accuracy   recall  precision F1_score     IoU
type0:  1928     0.061    0.061    0.998     0.929     0.939     0.959     0.891  


IndexError: list index out of range